In [1]:
import pandas as pd
import requests
import urllib3
from matplotlib import pyplot as plt
import seaborn as sns

activities = 'https://www.strava.com/api/v3/athlete/activities'
auth_url = "https://www.strava.com/oauth/token"

In [2]:
payload = {
    'client_id': "90629",
    'client_secret': 'dac647219a4e276d758fe38ffb52906e4d5144d8', 
    'refresh_token': '04119dc851c6419a8fbc4f0c205f672cbc1a1bc0',
    'grant_type': "refresh_token",
    'f': 'json'}

res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

print(res)

C:\Users\Eduardo\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.strava.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>


In [3]:
my_dataset = pd.DataFrame()
empty_page = True
page = 1
while empty_page:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': page}
    page_dataset = requests.get(activities, headers=header, params=param).json()
    my_dataset = pd.concat([my_dataset,pd.DataFrame(page_dataset)], ignore_index = True)
    empty_page = bool(page_dataset)
    if bool(page_dataset):
        print('\n Página:',page)
    else:
        print("\n Parando")
    page += 1
print("\n {} recordes carregados!".format(len(my_dataset)))


 Página: 1

 Página: 2

 Parando

 254 recordes carregados!


In [4]:
my_dataset['start_date'] = pd.to_datetime(my_dataset['start_date'],infer_datetime_format = True)


In [5]:
my_dataset[['type','distance']].groupby('type').agg('count')

,distance
type,
Hike,6
Ride,4
Run,222
Walk,21
Workout,1
